In [8]:
# IMPORTS GALORE
import pandas as pd
import numpy as np
import scipy as sp

import statsmodels.formula.api as sm
from sklearn import linear_model

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')   #seaborn-poster
plt.rcParams['figure.figsize'] = (11, 8)


import requests 

#activate if need arises:
#import warnings
#warnings.filterwarnings('ignore')

In [28]:
#local filepath, do not re-run cell or specify given file path on local machine 
file = '/Users/HendrikS/Documents/GitHub/Data-Madness/2017multipleChoiceResponses.csv'
import chardet
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [30]:
df_response17 = pd.read_csv('2017multipleChoiceResponses.csv',encoding = "ISO-8859-1", engine='python')
df_response18 = pd.read_csv('2018multipleChoiceResponses.csv',encoding = "ISO-8859-1", engine='python')
df_response19 = pd.read_csv('2019multipleChoiceResponses.csv',encoding = "ISO-8859-1", engine='python')

In [36]:
df_response17.loc[:,['Country','CompensationAmount']].groubby('Country') 

AttributeError: 'DataFrame' object has no attribute 'groubby'